In [ ]:
!pip install apache_beam[gcp]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 40.4 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
# Install system libraries
!apt-get update
!apt-get install -y libvips-dev
!pip install pyvips
!pip install openslide

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,363 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://security.ubu

In [ ]:
!pip install openslide_python.tar.gz

Processing ./openslide_python.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openslide-python: filename=openslide_python-1.4.2-cp311-abi3-linux_x86_64.whl size=35723 sha256=f9ef32dbe533afd153f2dd6ec0fba6cbcd0a112af9c388f62d46e2f9a999af74
  Stored in directory: /tmp/pip-ephem-wheel-cache-t9v19h4m/wheels/78/0b/fa/20c6d0ce3394f9ecee8b704115771252243222f068831a788d
Successfully built openslide-python


In [ ]:
import pandas as pd
import apache_beam as beam
import csv
import openslide
import os
from numpy import typing as npt
import numpy as np
import pyvips
import numpy as np
from typing import List, Tuple
import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

In [ ]:
#csv_path = '/kaggle/input/mayo-clinic-strip-ai/train.csv'
image_path = 'gs://mayo-clinic-data/006388_0.tif'
csv_path = 'gs://mayo-clinic-data/train.csv'
train_csv = pd.read_csv(csv_path)
train_csv

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE
...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE
750,fe9bec_0,4,fe9bec,0,LAA
751,ff14e0_0,6,ff14e0,0,CE
752,ffec5c_0,7,ffec5c,0,LAA


In [ ]:
list(train_csv.columns)

['image_id', 'center_id', 'patient_id', 'image_num', 'label']

In [ ]:
def key_patient_info(row_dict):
    return (row_dict['patient_id'], row_dict)

def _bytes_feature(value):
    if not isinstance(value, list):
        value =  [value]
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=value))

def _float_feature(value):
    if not isinstance(value, list):
        value =  [value]
    return tf.train.Feature(float_list = tf.train.FloatList(value=value))

def _int64_feature(value):
    if not isinstance(value, list):
        value =  [value]
    return tf.train.Feature(int64_list = tf.train.Int64List(value=value))

def load_downscaled_image(
    local_file_path: str,
    downsample_ratio: int = 16,
) -> npt.NDArray[np.uint8]:
    full_img = pyvips.Image.new_from_file(local_file_path)
    scaled_down_image: npt.NDArray[np.uint8] = full_img.resize(
        1 / downsample_ratio
    ).numpy()
    return scaled_down_image

def is_non_empty(tile: np.ndarray, background_threshold: float = 0.1) -> bool:
    """
    Determine if a tile is non-empty based on percentage of non-background pixels.
    """
    gray = np.mean(tile, axis=2)
    background_mask = gray > 190  # Adjust threshold as needed
    background_ratio = np.sum(background_mask) / tile.size
    print('Y/N', background_ratio < background_threshold)
    return background_ratio < background_threshold

class PatientDataCombineFn(beam.CombineFn):
    def create_accumulator(self):
        return {
            'center_ids': set(),
            'labels': set(),
            'image_ids': []
        }

    def add_input(self, acc, element):
        acc['center_ids'].add(element['center_id'])
        if 'label' in element:
            acc['labels'].add(element['label'])
        acc['image_ids'].append(element['image_id'])
        return acc

    def merge_accumulators(self, accs):
        result = self.create_accumulator()
        for acc in accs:
            result['center_ids'].update(acc['center_ids'])
            result['labels'].update(acc['labels'])
            result['image_ids'].extend(acc['image_ids'])
        return result

    def extract_output(self, acc):
        return {
            'center_id': list(acc['center_ids'])[0],
            'label': list(acc['labels'])[0] if acc['labels'] else None,
            'image_ids': acc['image_ids']
        }

class GenerateTiles(beam.DoFn):
    def __init__(self, tiff_files_location, tile_size=448, downsample_ratio=16):
        super().__init__()
        self.tiff_files_location = tiff_files_location
        self.downsample_ratio = downsample_ratio
        self.tile_size = tile_size
    def process(self, element):
        print(element)
        key, row = element
        image_id = row['image_id']
        image_path = os.path.join(self.tiff_files_location, f'{image_id}.tif')
        img = load_downscaled_image(image_path, downsample_ratio=self.downsample_ratio)
        height,width,channels = img.shape
        #tiles = [img[x:x+tile_size,y:y+N=tile_size] for x in range(0,height,tile_size) for y in range(0,width,tile_size)]
        #background_pixels = np.all(tile > 190, axis=2)
        #background_ratio = np.mean(background_pixels)
        for x in range(0, height, self.tile_size):
            for y in range(0, width, self.tile_size):
                tile = img[x:x+self.tile_size, y:y+self.tile_size]
                yield key, {'image_path':image_path, 'tile_info': (x,y,tile)}

class ExtractNonEmptyTiles(beam.DoFn):
    def __init__(self, tile_size=448, background_threshold=0.10):
        super().__init__()
        self.tile_size = tile_size
        self.background_threshold = background_threshold
    def process(self, element):
        key, ele_dict = element
        image_path = ele_dict['image_path']
        x,y,tile = ele_dict['tile_info']
        position = (y,x)
        size = (self.tile_size, self.tile_size)
        with openslide.open_slide(image_path) as slide:
            if is_non_empty(tile, background_threshold=self.background_threshold):
                tile_img = slide.read_region(position, 0, size).convert("RGB")
                tile = tf.convert_to_tensor(tile_img)
                encoded_tile = tf.io.encode_png(tile)
                yield key, encoded_tile.numpy()

class ProduceTiles(beam.PTransform):
    def __init__(self, tiff_files_location, downsample_ratio=16, tile_size=448, background_threshold=0.10):
        super().__init__()
        self.tiff_files_location = tiff_files_location
        self.downsample_ratio = downsample_ratio
        self.tile_size = tile_size
        self.background_threshold = background_threshold
    def expand(self, pcoll):
        return (pcoll
                | beam.ParDo(GenerateTiles(self.tiff_files_location))
                | beam.ParDo(ExtractNonEmptyTiles(background_threshold=self.background_threshold)))

class ConvertToTFRecords(beam.DoFn):
    def process(self, inputs):
        image_id = inputs[0]
        center_id = inputs['center_id']
        label = inputs['label']
        image_ids = inputs['image_ids']
        tiles = inputs['tiles']

        all_features = tf.train.Features(feature={
            'image_id': _bytes_feature(image_id),
            'center_id': _int64_feature(center_id),
            'label': _bytes_feature(label),
            'image_ids':  _bytes_feature(image_ids),
            'tiles': _bytes_feature(tiles)
        })
        example_proto = tf.train.Example(features=all_features)
        yield example_proto.SerializeToString()

options = {
    #'runner': 'DirectRunner',
    #'num_workers': 0,
    #'direct_running_mode': 'multiprocessing',
    'runner': 'DataflowRunner',
    'project': 'my-notebook-372506',
    'region': 'us-east1',
    'temp_location': 'gs://mayo-clinic-data/',
}

#with open(csv_path, 'r') as f:
#    header_line = f.readline().strip()
#    header = header_line.split(',')

header = list(train_csv.columns)
pipeline_options = beam.pipeline.PipelineOptions(flags=[], **options)
def run():


    with beam.Pipeline(options=pipeline_options) as p:
        csv_lines = (
            p
            | "ReadCSV" >> beam.io.ReadFromText(csv_path, skip_header_lines=1)
        )
        parsed_row_dicts = (
            csv_lines
            | beam.Map(lambda row: dict(zip(header, row.split(','))))
        )
        keyed_rows = (
            parsed_row_dicts
            | beam.Map(key_patient_info)
        )
        patient_info = (
            keyed_rows
            | beam.CombinePerKey(PatientDataCombineFn())
            #| beam.Map(print)
        )

        tiles = (
            keyed_rows
            | beam.combiners.Sample.FixedSizeGlobally(1)
            | beam.Map(lambda x: x[0])
            | ProduceTiles(tiff_files_location='gs://mayo-clinic-demo',
                          background_threshold=0.3)
        )

        merged_examples = ((patient_info, tiles)
                  | beam.CoGroupByKey()
                  | beam.Filter(lambda ele: ele[1][1])  #removes if list is empty of tiles is empty
                  | beam.MapTuple(lambda key, value: (key, {**value[0][0], 'tiles':value[1]}))
                 )
        merged_examples | beam.io.tfrecordio.WriteToTFRecord(file_path_prefix='gs://mayo-clinic-demo/tfrecords/output',
                                                            file_name_suffix='.tfrecord',
                                                            coder=beam.coders.BytesCoder())
run()

### Beam Practice